In [40]:
# !pip install higher tensorboard scipy networkx scikit-learn
import torch
from tqdm.notebook import tqdm
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision import datasets,transforms
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
import time
import torch.nn.functional as F
import copy
from torch import autograd
import higher
import itertools
import numpy as np
import scipy.sparse as sp
import torch
import networkx as nx
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score
import pickle
import math
import torch
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
import argparse
import scipy.sparse as sp
import numpy as np
import torch
import ipdb
from scipy.io import loadmat
import networkx as nx
import multiprocessing as mp
import torch.nn.functional as F
from functools import partial
import random
from sklearn.metrics import roc_auc_score, f1_score
from copy import deepcopy
from scipy.spatial.distance import pdist,squareform
import time
import argparse
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
import random
import ipdb
import copy
import torch.nn as nn
import torch.nn.functional as F
import math
import torch
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
from torch.nn import init
import argparse
import scipy.sparse as sp
import numpy as np
import torch
import ipdb
from scipy.io import loadmat
from collections import defaultdict
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import scipy.sparse as sp
import torch
import networkx as nx
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score
import pickle
import torch.nn.functional as F
import itertools as it
import matplotlib.pyplot as plt
import time
import argparse
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
import random
import ipdb
import copy
import argparse
import scipy.sparse as sp
import numpy as np
import torch
import ipdb
from scipy.io import loadmat
import networkx as nx
import multiprocessing as mp
import torch.nn.functional as F
from functools import partial
import random
from sklearn.metrics import roc_auc_score, f1_score
from copy import deepcopy
from scipy.spatial.distance import pdist,squareform
import torch.nn as nn
import torch.nn.functional as F
import math
import torch
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
from torch.nn import init
import ipdb
import argparse
import scipy.sparse as sp
import numpy as np
import torch
import ipdb
from scipy.io import loadmat
from collections import defaultdict
# torch.backends.cudnn.enabled = True
# torch.backends.cudnn.benchmark = True

GCN layer based on : https://arxiv.org/abs/1609.02907

In [26]:
# class GraphConvolution(Module):
#     def __init__(self, in_features, out_features, order, bias=True):
#         super(GraphConvolution, self).__init__()
#         self.in_features = in_features
#         self.out_features = out_features
#         self.order = order
#         self.weight = torch.nn.ParameterList([])
#         for i in range(self.order):
#             self.weight.append(Parameter(torch.FloatTensor(in_features, out_features)))
#         if bias:
#             self.bias = Parameter(torch.FloatTensor(out_features))
#         else:
#             self.register_parameter('bias', None)
#         self.reset_parameters()

#     def reset_parameters(self):
#         for i in range(self.order):
#             stdv = 1. / math.sqrt(self.weight[i].size(1))
#             self.weight[i].data.uniform_(-stdv, stdv)
#         if self.bias is not None:
#             self.bias.data.uniform_(-stdv, stdv)

#     def forward(self, input, adj_mtx):
#         output = []
#         if self.order == 1 and type(adj_mtx) != list:
#             adj = [adj_mtx]
#         for i in range(self.order):
#             support = torch.mm(input, self.weight[i])
#             # output.append(support)
#             output.append(torch.mm(adj_mtx[i], support))
#         output = sum(output)
#         if self.bias is not None:
#             return output + self.bias
#         else:
#             return output

#     def __repr__(self):
#         return self.__class__.__name__ + ' (' \
#                + str(self.in_features) + ' -> ' \
#                + str(self.out_features) + ')'

class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.DoubleTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.DoubleTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        #for 3_D batch, need a loop!!!


        if self.bias is not None:
            return output + self.bias
        else:
            return output


class GCN_Encoder(nn.Module):
    def __init__(self, nfeat, nhid, nembed, dropout):
        super(GCN_Encoder, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid)
        self.dropout = dropout

    def forward(self, x, adj, func=F.relu):
        x = func(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)

        return x

class GCN_Encoder2(nn.Module):
    def __init__(self, nfeat, nhid, nembed, dropout):
        super(GCN_Encoder2, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nembed)
        self.dropout = dropout

    def forward(self, x, adj, func=F.relu):
        x = func(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = func(self.gc2(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        return x

class GCN_Encoder3(nn.Module):
    def __init__(self, nfeat, nhid, nembed, dropout, nclass, order):
        super(GCN_Encoder3, self).__init__()

        layers = []
        if len(nhid) == 0:
            # layers.append(GraphConvolution(nfeat, nclass, order=order))
            layers.append(GraphConvolution(nfeat, nclass))
        else:
            # layers.append(GraphConvolution(nfeat, nhid[0], order=order))
            layers.append(GraphConvolution(nfeat, nhid[0]))
            for i in range(len(nhid) - 1):
                # layers.append(GraphConvolution(nhid[i], nhid[i + 1], order=order))
                layers.append(GraphConvolution(nhid[i], nhid[i + 1]))
        if nclass > 1:
            layers.append(GraphConvolution(nhid[-1], nembed))
            # layers.append(GraphConvolution(nhid[-1], nclass, order=order))
        self.gc = nn.ModuleList(layers)

        self.dropout = dropout
        self.nclass = nclass

    def forward(self, x, adj, func=F.relu):
        end_layer = len(self.gc) - 1 if self.nclass > 1 else len(self.gc)
        for i in range(end_layer):
            x = F.dropout(x, self.dropout, training=self.training)
            x = self.gc[i](x, adj)
            x = func(x)
        return x

class GCN_Classifier(nn.Module):
    def __init__(self, nembed, nhid, nclass, dropout, device):
        super(GCN_Classifier, self).__init__()
        self.device = device
        self.gc1 = GraphConvolution(nembed, nhid)
        self.mlp = nn.Linear(nhid, nclass, device=device).double()
        self.dropout = dropout

        self.reset_parameters()

    def reset_parameters(self):
        nn.init.normal_(self.mlp.weight,std=0.05)

    def forward(self, x, adj, func=F.relu):
        x = func(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.mlp(x)

        return x

def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

def norm_sparse(adj):#normalize a torch dense tensor for GCN, and change it into sparse.
    adj = adj + torch.eye(adj.shape[0]).to(adj)
    rowsum = torch.sum(adj,1)
    r_inv = 1/rowsum
    r_inv[torch.isinf(r_inv)] = 0.
    new_adj = torch.mul(r_inv.reshape(-1,1), adj)

    indices = torch.nonzero(new_adj).t()
    values = new_adj[indices[0], indices[1]] # modify this based on dimensionality
    return torch.sparse.DoubleTensor(indices, values, new_adj.size())

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.DoubleTensor(indices, values, shape)

def find_shown_index(adj, center_ind, steps = 2):
    seen_nodes = {}
    shown_index = []
    if isinstance(center_ind, int):
        center_ind = [center_ind]
    for center in center_ind:
        shown_index.append(center)
        if center not in seen_nodes:
            seen_nodes[center] = 1
    start_point = center_ind
    for step in range(steps):
        new_start_point = []
        candid_point = set(adj[start_point,:].reshape(-1, adj.shape[1]).nonzero()[:,1])
        for i, c_p in enumerate(candid_point):
            if c_p.item() in seen_nodes:
                pass
            else:
                seen_nodes[c_p.item()] = 1
                shown_index.append(c_p.item())
                new_start_point.append(c_p)
        start_point = new_start_point
    return shown_index

def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)
def print_class_acc(output, labels, pre='valid'):
    pre_num = 0
    #print class-wise performance
    '''
    for i in range(labels.max()+1):
        
        cur_tpr = accuracy(output[pre_num:pre_num+class_num_list[i]], labels[pre_num:pre_num+class_num_list[i]])
        print(str(pre)+" class {:d} True Positive Rate: {:.3f}".format(i,cur_tpr.item()))

        index_negative = labels != i
        labels_negative = labels.new(labels.shape).fill_(i)
        
        cur_fpr = accuracy(output[index_negative,:], labels_negative[index_negative])
        print(str(pre)+" class {:d} False Positive Rate: {:.3f}".format(i,cur_fpr.item()))

        pre_num = pre_num + class_num_list[i]
    '''
    #ipdb.set_trace()
    if labels.max() > 1:
        auc_score = roc_auc_score(labels.detach(), F.softmax(output, dim=-1).detach(), average='macro', multi_class='ovr')
    else:
        auc_score = roc_auc_score(labels.detach(), F.softmax(output, dim=-1)[:,1].detach(), average='macro')
    macro_F = f1_score(labels.detach(), torch.argmax(output, dim=-1).detach(), average='macro')
    print(str(pre)+' current auc-roc score: {:f}, current macro_F score: {:f}'.format(auc_score,macro_F))
    return

Initialization

In [27]:
weight_decay = 5e-4
epochs = 1000
learning_rate = 0.01
learning_rate_W = 0.01
dropout = 0.5
dropout_W = 0.5
gamma = 1
no_cuda = False
train_ratio=0.6
test_ratio=0.2
n_classes = 2
seed = 1234
torch.manual_seed(seed)
dataset = "diabetes"
order = 4
n_features = 0
w_val_size = 10
imbalance_ratio = None
n_hidden = 0
setting = None

Fixing the train set's imbalance ratio

In [28]:
# imbalance_ratio = 0.03
def fix_imbalance_ratio(imbalance_ratio, train_X, train_Y, train_idx, val_idx, test_idx):
    if imbalance_ratio != None:
        train_total_count = len(train_Y)
        train_labels_df = pd.DataFrame(train_Y, columns=['labels'])
        labels = pd.DataFrame(train_Y, columns=['labels']).labels.unique()
        train_grouped_df = train_labels_df.groupby(by=['labels'])['labels'].count()
        train_minority_count = train_grouped_df.min()
        train_minority_index = train_grouped_df.argmin()
        n_classes = len(train_grouped_df)
        concated_train = np.concatenate([train_X, train_Y[:, None]], axis=1)
        col_num = concated_train.shape[-1]
        grouped_con_df = pd.DataFrame(pd.DataFrame(concated_train).groupby(by=[col_num - 1]))
        current_minority_total_ratio = train_minority_count / train_total_count
        train_minority_label = grouped_con_df.iloc[train_minority_index, 0]
        assert current_minority_total_ratio > imbalance_ratio, "The ratio is below the threshold"
        rest_classes_count = train_total_count - train_minority_count
        reduction = int(imbalance_ratio * rest_classes_count * (imbalance_ratio - 1) + train_minority_count)
        minority_ndarray = grouped_con_df.iloc[train_minority_index, 1]
        drop_indices = np.random.choice(minority_ndarray.index, reduction, replace=False)
        new_train_X = train_X.drop(drop_indices)
        new_train_Y = train_Y.drop(drop_indices)
        train_idx = train_idx[: -reduction]
        val_idx = [x - reduction for x in val_idx]
        test_idx = [x - reduction for x in test_idx]
        return new_train_X, new_train_Y, train_idx, val_idx, test_idx
    return train_X, train_Y, train_idx, val_idx, test_idx

Generate the Pima Indian Diabetes (Diabetes) graph through the method used in : https://arxiv.org/abs/2103.00221

In [29]:
diabetes_df = pd.read_csv('./data/diabetes.csv')
# print(diabetes_df.head(10))
dataset_length = diabetes_df.values.shape[0]
diabetes_n_features = diabetes_df.values.shape[1] - 2
diabetes_train_X_df = diabetes_df.drop(columns=('Glucose')).values[:int(dataset_length * train_ratio), :-1]
diabetes_train_Y_df = diabetes_df.values[:int(dataset_length * train_ratio), -1]
diabetes_val_X_df = diabetes_df.drop(columns=('Glucose')).values[int(dataset_length * train_ratio):int(dataset_length * (1 - test_ratio)), :-1]
diabetes_val_Y_df = diabetes_df.values[int(dataset_length * train_ratio):int(dataset_length * (1 - test_ratio)), -1]
diabetes_test_X_df = diabetes_df.drop(columns=('Glucose')).values[int(dataset_length * (1 - test_ratio)):, :-1]
diabetes_test_Y_df = diabetes_df.values[int(dataset_length * (1 - test_ratio)):, -1]
diabetes_train_idx = list(range(int(dataset_length * train_ratio)))
diabetes_val_idx = list(range(int(dataset_length * train_ratio), int(dataset_length * (1 - test_ratio))))
diabetes_test_idx = list(range(int(dataset_length * (1 - test_ratio)), dataset_length))
diabetes_train_X_df, diabetes_train_Y_df, diabetes_train_idx, diabetes_val_idx, diabetes_test_idx = fix_imbalance_ratio(imbalance_ratio, \
    diabetes_train_X_df, diabetes_train_Y_df, diabetes_train_idx, diabetes_val_idx, diabetes_test_idx)
G = nx.Graph()
gam = 4
for patient_id, row_vals in diabetes_df.iterrows():
    G.add_node(str(patient_id), 
    pregnancies = row_vals[0], 
    # glucose = row_vals[1], 
    bloodpressure = row_vals[2], 
    skinthickness = row_vals[3], 
    insulin = row_vals[4], 
    bmi = row_vals[5], 
    diabetespedigreefunction = row_vals[6], 
    age = row_vals[7]) 
    # , outcome = row_vals[8])
# Two loops because of the order problem of NetworkX
for patient_id, row_vals in diabetes_df.iterrows():
    for other_patient_id in range(patient_id + 1, diabetes_df.shape[0]):
        if abs(diabetes_df.iloc[[other_patient_id], 1].values[0] - diabetes_df.iloc[[patient_id], 1].values[0]) < gam:
            G.add_edge(*(str(patient_id), str(other_patient_id)))
diabetes_adj_mtx = nx.to_numpy_matrix(G)
# print(diabetes_adj_mtx)
# print(G)
# pd.DataFrame(diabetes_adj_mtx).to_csv("data/diabetes_adj_mtx.csv")

Haberman’s survival (Haberman) graph through the method used in : https://arxiv.org/abs/2103.00221

In [30]:
haberman_df = pd.read_csv('./data/haberman.csv')
# print(diabetes_df.head(10))
dataset_length = haberman_df.values.shape[0]
haberman_n_features = haberman_df.values.shape[1] - 2
haberman_train_X_df = haberman_df.drop(columns=('Lymph Nodes')).values[:int(dataset_length * train_ratio), :-1]
haberman_train_Y_df = haberman_df.values[:int(dataset_length * train_ratio), -1]
haberman_val_X_df = haberman_df.drop(columns=('Lymph Nodes')).values[int(dataset_length * train_ratio):int(dataset_length * (1 - test_ratio)), :-1]
haberman_val_Y_df = haberman_df.values[int(dataset_length * train_ratio):int(dataset_length * (1 - test_ratio)), -1]
haberman_test_X_df = haberman_df.drop(columns=('Lymph Nodes')).values[int(dataset_length * (1 - test_ratio)):, :-1]
haberman_test_Y_df = haberman_df.values[int(dataset_length * (1 - test_ratio)):, -1]
haberman_train_idx = list(range(int(dataset_length * train_ratio)))
haberman_val_idx = list(range(int(dataset_length * train_ratio), int(dataset_length * (1 - test_ratio))))
haberman_test_idx = list(range(int(dataset_length * (1 - test_ratio)), dataset_length))
haberman_train_X_df, haberman_train_Y_df, haberman_train_idx, haberman_val_idx, haberman_test_idx = fix_imbalance_ratio(imbalance_ratio, \
    haberman_train_X_df, haberman_train_Y_df, haberman_train_idx, haberman_val_idx, haberman_test_idx)
G = nx.Graph()
gam = 2
for patient_id, row_vals in haberman_df.iterrows():
    G.add_node(str(patient_id),
        age = row_vals[0],
        operation_year = row_vals[1],
        lymph_nodes = row_vals[2],
    )
    # survival = row_vals[3])
for patient_id, row_vals in haberman_df.iterrows():
    for other_patient_id in range(patient_id + 1, haberman_df.shape[0]):
        if abs(haberman_df.iloc[[other_patient_id], 2].values[0] - haberman_df.iloc[[patient_id], 2].values[0]) < gam:
            G.add_edge(*(str(patient_id), str(other_patient_id)))
haberman_adj_mtx = nx.to_numpy_matrix(G)
# print(haberman_adj_mtx)
# print(G)
# pd.DataFrame(diabetes_adj_mtx).to_csv("data/haberman_adj_mtx.csv")

Sigmoid function

In [31]:
def sigmoid(mx, dim):
    mx = torch.sigmoid(mx)
    return F.normalize(mx, p=1, dim=dim)

Dataset specific variables

In [32]:
if dataset == "diabetes":
    adj_mtx = diabetes_adj_mtx
    n_hidden = [64, 64, 64]
    n_features = diabetes_n_features
    train_X = diabetes_train_X_df
    train_Y = diabetes_train_Y_df
    val_X = diabetes_val_X_df
    val_Y = diabetes_val_Y_df
    test_X = diabetes_test_X_df
    test_Y = diabetes_test_Y_df
    train_idx = diabetes_train_idx
    val_idx = diabetes_val_idx
    test_idx = diabetes_test_idx
elif dataset == "haberman":
    adj_mtx = haberman_adj_mtx
    n_hidden = [64]
    n_features = haberman_n_features
    train_X = haberman_train_X_df
    train_Y = haberman_train_Y_df
    val_X = haberman_val_X_df
    val_Y = haberman_val_Y_df
    test_X = haberman_test_X_df
    test_Y = haberman_test_Y_df
    train_idx = haberman_train_idx
    val_idx = haberman_val_idx
    test_idx = haberman_test_idx
else:
    pass

In [33]:
assert len(pd.DataFrame(val_Y, columns=['labels']).labels.unique()) == len(pd.DataFrame(train_Y, columns=['labels']).labels.unique()) == len(pd.DataFrame(test_Y, columns=['labels']).labels.unique()), \
    "There are some classes missing in one the 3 partitiones of the dataset"

Balanced sampling

In [34]:
def balanced_sampling(val_size):
    val_total_length = len(val_Y)
    labels_df = pd.DataFrame(val_Y, columns=['labels'])
    labels = pd.DataFrame(val_Y, columns=['labels']).labels.unique()
    val_grouped_df = labels_df.groupby(by=['labels'])['labels'].count()
    val_minority_count = val_grouped_df.min()
    assert val_minority_count > val_size, "The size of minority class is less the the sampling size"
    n_classes = len(val_grouped_df)
    concated_val = np.concatenate([val_X, val_Y[:, None]], axis=1)
    col_num = concated_val.shape[-1]
    grouped_con_df = pd.DataFrame(pd.DataFrame(concated_val).groupby(by=[col_num - 1]))
    balanced_val_set = pd.DataFrame(columns=[list(range(col_num))]).to_numpy()
    balanced_val_idx = []
    for label_index in range(n_classes):
        temp_df = pd.DataFrame(grouped_con_df.iloc[label_index, 1].sample(n = val_size))
        balanced_val_idx += list(temp_df.index)
        balanced_val_set = pd.DataFrame(np.concatenate([balanced_val_set, temp_df.to_numpy()], axis=0))
    return balanced_val_set, balanced_val_idx

The model

In [35]:
# class Wighted_GCN(nn.Module):
#     def __init__(self, n_feat, n_hid, n_class, dropout, order):
#         super(Wighted_GCN, self).__init__()
#         layers = []
#         if len(n_hid) == 0:
#             layers.append(GraphConvolution(n_feat, n_class, order=order))
#         else:
#             layers.append(GraphConvolution(n_feat, n_hid[0], order=order))
#             for i in range(len(n_hid) - 1):
#                 layers.append(GraphConvolution(n_hid[i], n_hid[i + 1], order=order))
#         if n_class > 1:
#             layers.append(GraphConvolution(n_hid[-1], n_class, order=order))
#         self.gc = nn.ModuleList(layers)
#         self.dropout = dropout
#         self.nclass = n_class

#     def forward(self, x, adj, samples=-1, func=F.relu):
#         end_layer = len(self.gc) - 1 if self.nclass > 1 else len(self.gc)
#         for i in range(end_layer):
#             x = F.dropout(x, self.dropout, training=self.training)
#             x = self.gc[i](x, adj)
#             x = func(x)

#         if self.nclass > 1:
#             classifier = self.gc[-1](x, adj)
#             classifier = F.log_softmax(classifier, dim=1)
#             return classifier[samples,:], x
#         else:
#             return None, x

In [36]:
device = torch.device("cuda" if False else "cpu")


Dataframe to Tensor transformation

In [37]:
features = torch.from_numpy(np.concatenate((train_X, val_X, test_X), axis=0)).to(device)
labels = torch.from_numpy(np.int64(np.concatenate((train_Y, val_Y, test_Y), axis=0))).to(device)
train_idx = torch.from_numpy(np.array(train_idx, dtype=np.int64)).to(device)
val_idx = torch.from_numpy(np.array(val_idx, dtype=np.int64)).to(device)
test_idx = torch.from_numpy(np.array(test_idx, dtype=np.int64)).to(device)
adj_mtx = torch.from_numpy(np.array(adj_mtx, dtype=np.float64)).to(device)

Classifier

In [38]:
encoder = GCN_Encoder3(nfeat=n_features,
        nhid=n_hidden,
        nembed=n_hidden[-1],
        dropout=dropout,
        nclass=n_classes,
        order=1)
classifier = GCN_Classifier(nembed=n_hidden[-1], 
        nhid=n_hidden[-1], 
        nclass=int(labels.max().item()) + 1, 
        dropout=dropout, device=device)
optimizer_en = optim.Adam(encoder.parameters(),
                       lr=learning_rate, weight_decay=weight_decay)
optimizer_cls = optim.Adam(classifier.parameters(),
                       lr=learning_rate, weight_decay=weight_decay)
def train(epoch):
        t = time.time()
        encoder.train()
        classifier.train()
        optimizer_en.zero_grad()
        optimizer_cls.zero_grad()
        embed = encoder(features, adj_mtx)
        output = classifier(embed, adj_mtx)
        if setting == 'reweight':
                weight = "STH"
                loss_train = F.cross_entropy(output[train_idx], labels[train_idx], weight=weight)
        else:
                loss_train = F.cross_entropy(output[train_idx], labels[train_idx])
        acc_train = accuracy(output[train_idx], labels[train_idx])
        loss_train.backward()
        optimizer_en.step()
        optimizer_cls.step()
        loss_val = F.cross_entropy(output[val_idx], labels[val_idx])
        acc_val = accuracy(output[val_idx], labels[val_idx])
        print_class_acc(output[val_idx], labels[val_idx])
        print('Epoch: {:05d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))
for epoch in range(5000):
        train(epoch)
        


valid current auc-roc score: 0.508848, current macro_F score: 0.257539
Epoch: 00001 loss_train: 2906810.5143 acc_train: 0.4109 loss_val: 3529018.6540 acc_val: 0.2857 time: 0.0200s
valid current auc-roc score: 0.508734, current macro_F score: 0.429630
Epoch: 00002 loss_train: 5035952.1409 acc_train: 0.6196 loss_val: 2812049.2851 acc_val: 0.7532 time: 0.0212s
valid current auc-roc score: 0.491039, current macro_F score: 0.461067
Epoch: 00003 loss_train: 1442071.9306 acc_train: 0.5870 loss_val: 876870.7173 acc_val: 0.6883 time: 0.0160s
valid current auc-roc score: 0.525181, current macro_F score: 0.286389
Epoch: 00004 loss_train: 3557569.9442 acc_train: 0.4022 loss_val: 3870262.0322 acc_val: 0.3052 time: 0.0170s
valid current auc-roc score: 0.534029, current macro_F score: 0.420881
Epoch: 00005 loss_train: 816228.1517 acc_train: 0.4957 loss_val: 995725.4717 acc_val: 0.4221 time: 0.0164s
valid current auc-roc score: 0.500567, current macro_F score: 0.445897
Epoch: 00006 loss_train: 577620.

GraphSMOTE's implementation

In [ ]:
def recon_upsample(embed, labels, idx_train, adj=None, portion=1.0, im_class_num=3):
    c_largest = labels.max().item()
    avg_number = int(idx_train.shape[0]/(c_largest+1))
    adj_new = None
    for i in range(im_class_num):
        chosen = idx_train[(labels==(c_largest-i))[idx_train]]
        num = int(chosen.shape[0]*portion)
        if portion == 0:
            c_portion = int(avg_number/chosen.shape[0])
            num = chosen.shape[0]
        else:
            c_portion = 1

        for j in range(c_portion):
            chosen = chosen[:num]

            chosen_embed = embed[chosen,:]
            distance = squareform(pdist(chosen_embed.cpu().detach()))
            np.fill_diagonal(distance,distance.max()+100)

            idx_neighbor = distance.argmin(axis=-1)
            
            interp_place = random.random()
            new_embed = embed[chosen,:] + (chosen_embed[idx_neighbor,:]-embed[chosen,:])*interp_place


            new_labels = labels.new(torch.Size((chosen.shape[0],1))).reshape(-1).fill_(c_largest-i)
            idx_new = np.arange(embed.shape[0], embed.shape[0]+chosen.shape[0])
            idx_train_append = idx_train.new(idx_new)

            embed = torch.cat((embed,new_embed), 0)
            labels = torch.cat((labels,new_labels), 0)
            idx_train = torch.cat((idx_train,idx_train_append), 0)

            if adj is not None:
                if adj_new is None:
                    adj_new = adj.new(torch.clamp_(adj[chosen,:] + adj[idx_neighbor,:], min=0.0, max = 1.0))
                else:
                    temp = adj.new(torch.clamp_(adj[chosen,:] + adj[idx_neighbor,:], min=0.0, max = 1.0))
                    adj_new = torch.cat((adj_new, temp), 0)

    if adj is not None:
        add_num = adj_new.shape[0]
        new_adj = adj.new(torch.Size((adj.shape[0]+add_num, adj.shape[0]+add_num))).fill_(0.0)
        new_adj[:adj.shape[0], :adj.shape[0]] = adj[:,:]
        new_adj[adj.shape[0]:, :adj.shape[0]] = adj_new[:,:]
        new_adj[:adj.shape[0], adj.shape[0]:] = torch.transpose(adj_new, 0, 1)[:,:]

        return embed, labels, idx_train, new_adj.detach()

    else:
        return embed, labels, idx_train

train & eval

In [ ]:
import numpy as np
import scipy.sparse as sp
import torch
import networkx as nx
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score
import pickle


def encode_onehot_torch(labels):
    num_classes = int(labels.max() + 1)
    y = torch.eye(num_classes)
    return y[labels]


def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot